**Variance Reduction via Control Variates**

This notebook applies the Control Variate technique to enhance Monte Carlo pricing efficiency for Lookback options.
By leveraging a correlated instrument with known expected value (a vanilla call), we reduce sampling variance in the Lookback payoff estimate.
The method exploits covariance between simulated payoffs, improving estimator stability and achieving faster convergence to the true option price.

In [1]:
import numpy as np
from scipy.stats import norm

# --- Monte Carlo GBM Simulation ---
def mc_sim(S0, sigma, T, r, num_sim, num_steps):
    dt = T / num_steps
    paths = np.zeros((num_sim, num_steps))
    for i in range(num_sim):
        paths[i, 0] = S0
        for j in range(1, num_steps):
            z = np.random.randn()
            paths[i, j] = paths[i, j-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*z)
    return paths

# --- Lookback Payoff ---
def lookback_price(S0, sigma, T, r, num_sim, num_steps):
    paths = mc_sim(S0, sigma, T, r, num_sim, num_steps)
    return np.array([paths[i, -1] - np.min(paths[i]) for i in range(num_sim)])

# --- Vanilla Call Payoff ---
def vanilla_call(S0, K, sigma, T, r, num_sim, num_steps):
    paths = mc_sim(S0, sigma, T, r, num_sim, num_steps)
    return np.array([np.exp(-r*T)*max(paths[i, -1] - K, 0) for i in range(num_sim)])

# --- Control Variate Estimator ---
def control_variate(S0, K, sigma, T, r, num_sim, num_steps):
    X = lookback_price(S0, sigma, T, r, num_sim, num_steps)
    Y = vanilla_call(S0, K, sigma, T, r, num_sim, num_steps)
    EY = np.mean(Y)
    cov_XY = np.cov(X, Y)[0, 1]
    var_Y = np.var(Y)
    c = -cov_XY / var_Y
    ctrl_estimator = X + c * (Y - EY)
    return np.mean(ctrl_estimator), np.std(ctrl_estimator)

# --- Simulation Parameters ---
S0, K, sigma, T, r = 80, 70, 0.1, 1, 0.05
num_sim, num_steps = 500, 100

mean_ctrl, std_ctrl = control_variate(S0, K, sigma, T, r, num_sim, num_steps)
z = norm.ppf(0.99875)
margin = z * std_ctrl / np.sqrt(num_sim)
print(f"Control Variate 99.75% CI: [{mean_ctrl - margin:.4f}, {mean_ctrl + margin:.4f}]")


Control Variate 99.75% CI: [7.1315, 8.6896]
